In [1]:
# # Power BI Fabric Metadata Extraction Pipeline

# ## Overview
# # This notebook extracts comprehensive metadata from Microsoft Fabric workspaces including semantic models, reports, measures, tables, columns, relationships, security roles, and refresh history.

# ## Prerequisites
# - Access to Microsoft Fabric workspace
# - Required Python packages: `sempy`, `duckdb`, `pandas`, `pytz`

# ## Installation
# Run the following cell to install required dependencies:

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 3, Finished, Available, Finished)

In [2]:
!pip install duckdb -q

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 4, Finished, Available, Finished)

In [3]:
# Import required libraries
import sempy.fabric as fabric
import duckdb
import pandas as pd
import pytz
import re
from datetime import datetime, timedelta
from notebookutils import mssparkutils

# Configure timezone and date parameters
vn_tz = pytz.timezone('Asia/Ho_Chi_Minh')
time_now = datetime.now(vn_tz)
Previous_Date = (time_now - timedelta(days=14)).strftime("%Y-%m-%d")
Current_Date = time_now.strftime("%Y-%m-%d")
Current_Datetime = time_now.strftime("%Y-%m-%d %H:%M:%S")
Current_MonthID = int(time_now.strftime("%Y%m"))

print(f"📅 Extraction Period: {Previous_Date} to {Current_Date}")
print(f"🕐 Current Timestamp: {Current_Datetime}")
print(f"📊 Month ID: {Current_MonthID}")

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 5, Finished, Available, Finished)

📅 Extraction Period: 2025-11-09 to 2025-11-23
🕐 Current Timestamp: 2025-11-23 21:11:14
📊 Month ID: 202511


## Configuration & Initialization

This section imports required libraries and sets up timezone configurations for Vietnam (UTC+7). It establishes date ranges for data extraction:
- **Previous_Date**: 14 days before current date (for refresh history)
- **Current_Date**: Today's date
- **Current_Datetime**: Current timestamp for audit trails
- **Current_MonthID**: Year-month identifier (YYYYMM format)

## 1. Semantic Model Metadata Extraction

### Purpose
Extract metadata about all semantic models (datasets) in specified workspaces.

### Output Fields
- **YearMonth**: Reporting period identifier
- **WorkspaceID**: Unique workspace identifier
- **WorkspaceName**: Human-readable workspace name
- **DatasetID**: Unique semantic model identifier
- **DatasetName**: Semantic model name
- **CreatedTimestamp**: Dataset creation date
- **LastUpdate**: Last modification timestamp
- **ImportTime**: Metadata extraction timestamp

### Business Value
Provides inventory of all semantic models for governance, usage tracking, and lifecycle management.

In [4]:
# Define target workspaces
select_workspace = ["Fabric & Dagster"]

# Get workspace information
workspaces = fabric.list_workspaces()
workspaces = workspaces[workspaces['Name'].isin(select_workspace)].reset_index(drop=True)

print(f"🔍 Processing {len(workspaces)} workspace(s): {', '.join(workspaces['Name'].tolist())}")

# Extract semantic model metadata
metadata_semantic_model = pd.DataFrame()

for idx, (_, row) in enumerate(workspaces.iterrows(), 1):
    workspace_id = row['Id']
    workspace_name = row['Name']
    
    print(f"  [{idx}] Extracting from '{workspace_name}'...")
    
    semantic_model = fabric.list_datasets(workspace=workspace_name)
    
    # Transform data using DuckDB for efficient processing
    semantic_model = duckdb.query(f'''
        SELECT
            {Current_MonthID} AS YearMonth,
            '{workspace_id}' AS WorkspaceID,
            '{workspace_name}' AS WorkspaceName,
            "Dataset ID" AS DatasetID,
            "Dataset Name" AS DatasetName,
            "Created Timestamp" AS CreatedTimestamp,
            "Last Update" AS LastUpdate,
            CAST('{Current_Datetime}' AS TIMESTAMP) AS ImportTime
        FROM semantic_model
    ''').to_df()
    
    metadata_semantic_model = pd.concat([metadata_semantic_model, semantic_model], ignore_index=True)

print(f"✅ Extracted {len(metadata_semantic_model)} semantic model(s)\n")
metadata_semantic_model.head(5)

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 6, Finished, Available, Finished)

🔍 Processing 1 workspace(s): Fabric & Dagster
  [1] Extracting from 'Fabric & Dagster'...
✅ Extracted 3 semantic model(s)



,YearMonth,WorkspaceID,WorkspaceName,DatasetID,DatasetName,CreatedTimestamp,LastUpdate,ImportTime
0,202511,abf1b960-d04c-4434-9a4d-a84d961dbdad,Fabric & Dagster,1b46bd95-8778-4ed7-a36c-4582c4dbae62,FabricDagsterK1,2025-09-28 13:48:07,NaT,2025-11-23 21:11:14
1,202511,abf1b960-d04c-4434-9a4d-a84d961dbdad,Fabric & Dagster,27bee6e7-ed0b-44d0-a80b-d64090e7ddc4,FabircDagsterK1-Cloud,2021-02-12 23:00:58,NaT,2025-11-23 21:11:14
2,202511,abf1b960-d04c-4434-9a4d-a84d961dbdad,Fabric & Dagster,26ce789d-54a1-4ded-bdc9-de65897d2f8e,Dashboard,2021-02-12 23:00:58,NaT,2025-11-23 21:11:14


## 2. Report Metadata Extraction

### Purpose
Extract metadata about Power BI reports and link them to their underlying semantic models.

### Output Fields
- **YearMonth**: Reporting period
- **WorkspaceName**: Workspace containing the report
- **DatasetID/DatasetName**: Associated semantic model
- **DatasetCreatedTime**: When the underlying dataset was created
- **LastUpdate**: Dataset last refresh time
- **ReportID/ReportName**: Report identifiers
- **WebUrl**: Direct link to view report
- **EmbedUrl**: URL for embedding report
- **ImportTime**: Metadata extraction timestamp

### Business Value
Maps reports to datasets for impact analysis, access tracking, and content governance.

In [5]:
metadata_report = pd.DataFrame()

for idx, (_, row) in enumerate(workspaces.iterrows(), 1):
    workspace_id = row['Id']
    workspace_name = row['Name']
    
    print(f"  [{idx}] Extracting reports from '{workspace_name}'...")
    
    list_reports = fabric.list_reports(workspace=workspace_name)
    
    # Join reports with semantic models
    power_bi_reports = duckdb.query(f'''
        SELECT
            {Current_MonthID} AS YearMonth,
            b.WorkspaceName,
            b.DatasetID,
            b.DatasetName, 
            b.CreatedTimestamp AS DatasetCreatedTime,
            b.LastUpdate,
            a.Id AS ReportID,
            a.Name AS ReportName,
            a."Web Url" AS WebUrl,
            a."Embed Url" AS EmbedUrl,
            CAST('{Current_Datetime}' AS TIMESTAMP) AS ImportTime
        FROM list_reports a 
        LEFT JOIN metadata_semantic_model b 
            ON a."Dataset Id" = b.DatasetID
    ''').to_df()
    
    metadata_report = pd.concat([metadata_report, power_bi_reports], ignore_index=True)

print(f"✅ Extracted {len(metadata_report)} report(s)\n")
metadata_report.head(5)

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 7, Finished, Available, Finished)

  [1] Extracting reports from 'Fabric & Dagster'...
✅ Extracted 4 report(s)



,YearMonth,WorkspaceName,DatasetID,DatasetName,DatasetCreatedTime,LastUpdate,ReportID,ReportName,WebUrl,EmbedUrl,ImportTime
0,202511,Fabric & Dagster,1b46bd95-8778-4ed7-a36c-4582c4dbae62,FabricDagsterK1,2025-09-28 13:48:07,NaT,b7ee4aa5-3171-4889-b26d-a04318743915,FabricDagsterK1,https://app.powerbi.com/groups/abf1b960-d04c-4...,https://app.powerbi.com/reportEmbed?reportId=b...,2025-11-23 21:11:14
1,202511,Fabric & Dagster,27bee6e7-ed0b-44d0-a80b-d64090e7ddc4,FabircDagsterK1-Cloud,2021-02-12 23:00:58,NaT,c2e4d7a0-2fda-42e6-9f61-c3a13bb5fed8,Fabirc & Dagster K1 - Cloud,https://app.powerbi.com/groups/abf1b960-d04c-4...,https://app.powerbi.com/reportEmbed?reportId=c...,2025-11-23 21:11:14
2,202511,Fabric & Dagster,26ce789d-54a1-4ded-bdc9-de65897d2f8e,Dashboard,2021-02-12 23:00:58,NaT,7c5ca77a-03aa-4380-a96d-ca1d688d1d0d,Dashboard,https://app.powerbi.com/groups/abf1b960-d04c-4...,https://app.powerbi.com/reportEmbed?reportId=7...,2025-11-23 21:11:14
3,202511,None,None,None,NaT,NaT,189091a9-74e8-434d-a5ea-4bcb9fed68ab,Report Usage Metrics Report,https://app.powerbi.com/groups/abf1b960-d04c-4...,https://app.powerbi.com/reportEmbed?reportId=1...,2025-11-23 21:11:14


## 3. Measures Metadata Extraction

### Purpose
Extract all DAX measures from semantic models for documentation and lineage analysis.

### Output Fields
- **YearMonth**: Reporting period
- **DatasetID**: Parent semantic model
- **Measure**: Measure name
- **Expression**: DAX formula
- **Description**: Measure documentation
- **Display Folder**: Organization folder
- **Format String**: Number/date formatting
- **Data Category**: Semantic type
- **ImportTime**: Extraction timestamp

### Business Value
Enables measure documentation, DAX formula auditing, and business logic tracking across models.

In [21]:
metadata_measures = pd.DataFrame()
total_measures = 0

for idx, (_, row) in enumerate(metadata_semantic_model.iterrows(), 1):
    workspace_name = row["WorkspaceName"]
    dataset_name = row["DatasetName"]
    dataset_id = row["DatasetID"]
    
    print(f"  [{idx}/{len(metadata_semantic_model)}] Processing measures in '{dataset_name}'...")
    
    measures = fabric.list_measures(workspace=workspace_name, dataset=dataset_name)
    
    if len(measures) > 0:
        measures = duckdb.query(f'''
            SELECT
                {Current_MonthID} AS YearMonth,
                '{dataset_id}' AS DatasetID,
                a.*,
                CAST('{Current_Datetime}' AS TIMESTAMP) AS ImportTime
            FROM measures a 
        ''').to_df()
        measures.columns = [re.sub(r' ', '', col) for col in measures.columns]
        metadata_measures = pd.concat([metadata_measures, measures], ignore_index=True)
        total_measures += len(measures)

print(f"✅ Extracted {total_measures} measure(s) from {len(metadata_semantic_model)} dataset(s)\n")
metadata_measures.head(5)

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 23, Finished, Available, Finished)

  [1/3] Processing measures in 'FabricDagsterK1'...
  [2/3] Processing measures in 'FabircDagsterK1-Cloud'...
  [3/3] Processing measures in 'Dashboard'...
✅ Extracted 9 measure(s) from 3 dataset(s)



,YearMonth,DatasetID,TableName,MeasureName,MeasureExpression,MeasureDataType,MeasureHidden,MeasureDisplayFolder,MeasureDescription,FormatString,DataCategory,DetailRowsDefinition,FormatStringDefinition,ImportTime
0,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,All Measures,Measure,123,Int64,False,,,0,,<NA>,<NA>,2025-11-23 21:11:14
1,202511,27bee6e7-ed0b-44d0-a80b-d64090e7ddc4,All Measures,Salas Amount,"SUMX(F_Sales,[Amount])",Double,False,,,"#,0",,<NA>,<NA>,2025-11-23 21:11:14
2,202511,27bee6e7-ed0b-44d0-a80b-d64090e7ddc4,All Measures,Sales Target,"SUMX(F_Target,[Target])",Int64,False,,,"#,0",,<NA>,<NA>,2025-11-23 21:11:14
3,202511,27bee6e7-ed0b-44d0-a80b-d64090e7ddc4,All Measures,Openning Inventory,"SUMX(F_Inventory,[OpeningInventory])",Int64,False,,,"#,0",,<NA>,<NA>,2025-11-23 21:11:14
4,202511,27bee6e7-ed0b-44d0-a80b-d64090e7ddc4,All Measures,Closing Inventory,"SUMX(F_Inventory,[RemainingInventory])",Int64,False,,,"#,0",,<NA>,<NA>,2025-11-23 21:11:14


## 4. Table & Column Metadata Extraction

### Purpose
Extract comprehensive schema information including tables, columns, and their properties.

### Output Fields
- **YearMonth**: Reporting period
- **DatasetID**: Parent semantic model
- **TableName/TableType**: Table identification and type (Import/DirectQuery/Calculated)
- **TableQuery**: M/SQL query definition for the table
- **TableHidden/TableDescription**: Visibility and documentation
- **ColumnName/DataType**: Column schema information
- **ColumnHidden/ColumnDescription**: Column visibility and documentation
- **FormatString**: Display formatting
- **Source**: Column source expression
- **DisplayFolder**: Organizational structure
- **ModifiedTime**: Last modification timestamp
- **ImportTime**: Extraction timestamp

### Business Value
Provides complete data model schema for documentation, impact analysis, and data catalog integration.

In [7]:
metadata_table_columns = pd.DataFrame()
total_columns = 0

for idx, (_, row) in enumerate(metadata_semantic_model.iterrows(), 1):
    workspace_name = row["WorkspaceName"]
    dataset_name = row["DatasetName"]
    dataset_id = row["DatasetID"]
    
    print(f"  [{idx}/{len(metadata_semantic_model)}] Processing schema of '{dataset_name}'...")
    
    tables = fabric.list_tables(workspace=workspace_name, dataset=dataset_name)
    
    if len(tables) > 0:
        # Get additional table metadata via DAX
        temp_tables = fabric.evaluate_dax(
            workspace=workspace_name, 
            dataset=dataset_name, 
            dax_string="EVALUATE INFO.TABLES()"
        )
        partitions = fabric.evaluate_dax(
            workspace=workspace_name, 
            dataset=dataset_name, 
            dax_string="EVALUATE INFO.PARTITIONS()"
        )
        
        # Clean column names
        temp_tables.columns = [re.sub(r'[\[\]]', '', col) for col in temp_tables.columns]
        partitions.columns = [re.sub(r'[\[\]]', '', col) for col in partitions.columns]
        
        # Get column metadata
        columns = fabric.list_columns(workspace=workspace_name, dataset=dataset_name)
        
        # Combine all metadata
        data_tables_columns = duckdb.query(f'''
            SELECT
                {Current_MonthID} AS YearMonth,
                '{dataset_id}' AS DatasetID,
                b.Name AS TableName,
                b.Type AS TableType,
                d.QueryDefinition AS TableQuery,
                b.Hidden AS TableHidden,
                b.Description AS TableDescription,
                a."Column Name" AS ColumnName,
                a.Type AS ColumnType, 
                a.Hidden AS ColumnHidden,
                a.Description AS ColumnDescription,
                a."Data Type" AS DataType,
                a."Format String" AS FormatString,
                a.Source,
                a."Display Folder" AS DisplayFolder,
                a."Modified Time" AS ModifiedTime,
                CAST('{Current_Datetime}' AS TIMESTAMP) AS ImportTime
            FROM columns a 
            LEFT JOIN tables b ON a."Table Name" = b.Name
            LEFT JOIN temp_tables c ON b.Name = c.Name
            LEFT JOIN partitions d ON c.ID = d.TableID
        ''').to_df()
        
        metadata_table_columns = pd.concat([metadata_table_columns, data_tables_columns], ignore_index=True)
        total_columns += len(columns)

print(f"✅ Extracted {total_columns} column(s) from {len(tables)} table(s)\n")
metadata_table_columns.head(5)

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 9, Finished, Available, Finished)

  [1/3] Processing schema of 'FabricDagsterK1'...
  [2/3] Processing schema of 'FabircDagsterK1-Cloud'...
  [3/3] Processing schema of 'Dashboard'...
✅ Extracted 220 column(s) from 10 table(s)



,YearMonth,DatasetID,TableName,TableType,TableQuery,TableHidden,TableDescription,ColumnName,ColumnType,ColumnHidden,ColumnDescription,DataType,FormatString,Source,DisplayFolder,ModifiedTime,ImportTime
0,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,D_Customer,Table,"let\n Source = Sql.Database(""DESKTOP-P4K1FP...",False,,CustomerID,Data,False,,Int64,0,CustomerID,,2025-09-28 13:51:42,2025-11-23 21:11:14
1,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,D_Customer,Table,"let\n Source = Sql.Database(""DESKTOP-P4K1FP...",False,,CustomerName,Data,False,,String,,CustomerName,,2025-09-28 13:48:58,2025-11-23 21:11:14
2,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,D_Customer,Table,"let\n Source = Sql.Database(""DESKTOP-P4K1FP...",False,,Email,Data,False,,String,,Email,,2025-09-28 13:48:58,2025-11-23 21:11:14
3,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,D_Customer,Table,"let\n Source = Sql.Database(""DESKTOP-P4K1FP...",False,,BirthDate,Data,False,,DateTime,General Date,BirthDate,,2025-09-28 13:48:58,2025-11-23 21:11:14
4,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,D_Customer,Table,"let\n Source = Sql.Database(""DESKTOP-P4K1FP...",False,,Gender,Data,False,,String,,Gender,,2025-09-28 13:48:58,2025-11-23 21:11:14


## 5. Relationship Metadata Extraction

### Purpose
Extract table relationships to understand data model structure and dependencies.

### Output Fields
- **YearMonth**: Reporting period
- **DatasetID**: Parent semantic model
- **From Table/Column**: Source side of relationship
- **To Table/Column**: Target side of relationship
- **Cross Filtering Behavior**: Single/Both direction filtering
- **Cardinality**: One-to-One, One-to-Many, Many-to-One
- **Is Active**: Whether relationship is active
- **Security Filtering Behavior**: How RLS applies
- **ImportTime**: Extraction timestamp

### Business Value
Enables data lineage visualization, relationship validation, and impact analysis for model changes.

In [24]:
metadata_relationships = pd.DataFrame()
total_relationships = 0

for idx, (_, row) in enumerate(metadata_semantic_model.iterrows(), 1):
    workspace_name = row["WorkspaceName"]
    dataset_name = row["DatasetName"]
    dataset_id = row["DatasetID"]
    
    print(f"  [{idx}/{len(metadata_semantic_model)}] Extracting relationships from '{dataset_name}'...")
    
    relationships = fabric.list_relationships(workspace=workspace_name, dataset=dataset_name)
 
    if len(relationships) > 0:
        data_relationships = duckdb.query(f'''
            SELECT
                {Current_MonthID} AS YearMonth,
                '{dataset_id}' AS DatasetID,
                *,
                CAST('{Current_Datetime}' AS TIMESTAMP) AS ImportTime
            FROM relationships
        ''').to_df()
        data_relationships.columns = [re.sub(r' ', '', col) for col in data_relationships.columns]
        metadata_relationships = pd.concat([metadata_relationships, data_relationships], ignore_index=True)
        total_relationships += len(relationships)

print(f"✅ Extracted {total_relationships} relationship(s)\n")
metadata_relationships.head(5)

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 26, Finished, Available, Finished)

  [1/3] Extracting relationships from 'FabricDagsterK1'...
  [2/3] Extracting relationships from 'FabircDagsterK1-Cloud'...
  [3/3] Extracting relationships from 'Dashboard'...
✅ Extracted 22 relationship(s)



,YearMonth,DatasetID,Multiplicity,FromTable,FromColumn,ToTable,ToColumn,Active,CrossFilteringBehavior,SecurityFilteringBehavior,JoinOnDateBehavior,RelyOnReferentialIntegrity,State,ModifiedTime,RelationshipName,ImportTime
0,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,m:1,F_Target,OutletID,D_Outlet,OutletID,True,OneDirection,OneDirection,DateAndTime,False,Ready,2025-09-28 13:51:08,4ba24d1f-f14a-6fda-f608-5d5371106111,2025-11-23 21:11:14
1,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,m:1,F_Inventory,OutletID,D_Outlet,OutletID,True,OneDirection,OneDirection,DateAndTime,False,Ready,2025-09-28 13:51:12,9aefa038-948b-9f22-7e76-ef7ffde5b26e,2025-11-23 21:11:14
2,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,m:1,F_Sales,OutletID,D_Outlet,OutletID,True,OneDirection,OneDirection,DateAndTime,False,Ready,2025-09-28 13:51:23,a2e0fcd8-3809-7708-f561-79af364e1354,2025-11-23 21:11:14
3,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,m:1,F_Sales,CustomerID,D_Customer,CustomerID,True,OneDirection,OneDirection,DateAndTime,False,Ready,2025-09-28 13:51:42,5a6fd39c-92cd-8201-29fe-a73edc669118,2025-11-23 21:11:14
4,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62,m:1,F_Sales,StaffID,D_Staff,StaffID,True,OneDirection,OneDirection,DateAndTime,False,Ready,2025-09-28 13:52:00,9c3c51da-f5f0-8c71-0bb8-693c8db7fb91,2025-11-23 21:11:14


## 6. Role & Member Metadata Extraction

### Purpose
Extract Row-Level Security (RLS) roles and their member assignments for security auditing.

### Output Fields
- **YearMonth**: Reporting period
- **RoleKey**: Unique role identifier (DatasetID-RoleID)
- **DatasetID**: Parent semantic model
- **RoleID**: Internal role identifier
- **RoleName**: Human-readable role name
- **RoleDescription**: Role purpose documentation
- **MemberName**: User/Group assigned to role
- **ImportTime**: Extraction timestamp

### Business Value
Enables security auditing, access governance, and compliance reporting for data access controls.

In [34]:
metadata_role_members = pd.DataFrame()
total_members = 0

for idx, (_, row) in enumerate(metadata_semantic_model.iterrows(), 1):
    workspace_name = row["WorkspaceName"]
    dataset_name = row["DatasetName"]
    dataset_id = row["DatasetID"]
    
    print(f"  [{idx}/{len(metadata_semantic_model)}] Extracting roles from '{dataset_name}'...")
    
    try:
        # Query role definitions
        roles = fabric.evaluate_dax(
            workspace=workspace_name,
            dataset=dataset_name,
            dax_string="SELECT * FROM $SYSTEM.TMSCHEMA_ROLES"
        )
        
        # Query role memberships
        members = fabric.evaluate_dax(
            workspace=workspace_name,
            dataset=dataset_name,
            dax_string="SELECT * FROM $SYSTEM.TMSCHEMA_ROLE_MEMBERSHIPS"
        )
        
        # Combine role and member information
        data_role_members = duckdb.query(f'''
            SELECT
                {Current_MonthID} AS YearMonth,
                '{dataset_id}' || '-' || CAST(a.RoleID AS VARCHAR) AS RoleKey,
                '{dataset_id}' AS DatasetID,
                a.RoleID,
                b.Name AS RoleName,
                COALESCE(b.Description, 'No Description') AS RoleDescription,
                a.MemberName,
                CAST('{Current_Datetime}' AS TIMESTAMP) AS ImportTime
            FROM members a 
            LEFT JOIN roles b ON a.RoleID = b.ID
        ''').to_df()
        
        metadata_role_members = pd.concat([metadata_role_members, data_role_members], ignore_index=True)
        total_members += len(members)
        
    except Exception as e:
        print(f"    ⚠️  No roles found or access denied: {str(e)[:50]}")

print(f"✅ Extracted {total_members} role member(s)\n")
metadata_role_members.head(5)

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 36, Finished, Available, Finished)

  [1/3] Extracting roles from 'FabricDagsterK1'...
  [2/3] Extracting roles from 'FabircDagsterK1-Cloud'...
  [3/3] Extracting roles from 'Dashboard'...
✅ Extracted 2 role member(s)



,YearMonth,RoleKey,DatasetID,RoleID,RoleName,RoleDescription,MemberName,ImportTime
0,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62-3978,1b46bd95-8778-4ed7-a36c-4582c4dbae62,3978,View All,No Description,tu.thai.tram@8rc8vk.onmicrosoft.com,2025-11-23 21:11:14
1,202511,1b46bd95-8778-4ed7-a36c-4582c4dbae62-3978,1b46bd95-8778-4ed7-a36c-4582c4dbae62,3978,View All,No Description,hoang.huy.nguyen@8rc8vk.onmicrosoft.com,2025-11-23 21:11:14


## 7. RLS Expression Extraction

### Purpose
Extract Row-Level Security (RLS) filter expressions to document data access restrictions.

### Output Fields
- **YearMonth**: Reporting period
- **WorkspaceID/WorkspaceName**: Workspace context
- **DatasetID/DatasetName**: Parent semantic model
- **RoleKey**: Unique role identifier
- **RoleID/RoleName**: Role identification
- **TableName**: Table with RLS filter
- **FilterExpression**: DAX filter expression
- **ImportTime**: Extraction timestamp

### Business Value
Documents security logic for compliance, enables security auditing, and supports RLS testing/validation.

In [10]:
row_level_security_expression = pd.DataFrame()
total_rls_rules = 0

for idx, (_, row) in enumerate(metadata_semantic_model.iterrows(), 1):
    workspace_id = row['WorkspaceID']
    workspace_name = row["WorkspaceName"]
    dataset_name = row["DatasetName"]
    dataset_id = row["DatasetID"]
    
    print(f"  [{idx}/{len(metadata_semantic_model)}] Extracting RLS expressions from '{dataset_name}'...")
    
    try:
        rls_expression = fabric.get_row_level_security_permissions(
            workspace=workspace_name, 
            dataset=dataset_name
        )
        
        if len(rls_expression) > 0:
            # Get role information
            role_member = fabric.evaluate_dax(
                workspace=workspace_name, 
                dataset=dataset_name,
                dax_string='EVALUATE INFO.ROLES()'
            )
            role_member.columns = [re.sub(r'[\[\]]', '', col) for col in role_member.columns]
            
            # Combine RLS expressions with role metadata
            data_rls = duckdb.query(f'''
                WITH list_role_members AS (
                    SELECT DISTINCT
                        '{dataset_id}' || '-' || CAST(a.ID AS VARCHAR) AS RoleKey,
                        '{dataset_id}' AS DatasetID,
                        a.ID AS RoleID,
                        a.Name AS RoleName
                    FROM role_member a
                )
                SELECT DISTINCT
                    {Current_MonthID} AS YearMonth,
                    '{workspace_id}' AS WorkspaceID,
                    '{workspace_name}' AS WorkspaceName,
                    '{dataset_id}' AS DatasetID,
                    '{dataset_name}' AS DatasetName,
                    b.RoleKey, 
                    b.RoleID, 
                    b.RoleName,
                    a.Table AS TableName,
                    a."Filter Expression" AS FilterExpression,
                    CAST('{Current_Datetime}' AS TIMESTAMP) AS ImportTime
                FROM rls_expression a 
                LEFT JOIN list_role_members b ON a.Role = b.RoleName 
            ''').to_df()
            
            row_level_security_expression = pd.concat([row_level_security_expression, data_rls], ignore_index=True)
            total_rls_rules += len(data_rls)
            
    except Exception as e:
        print(f"    ⚠️  No RLS found or access denied: {str(e)[:50]}")

print(f"✅ Extracted {total_rls_rules} RLS expression(s)\n")
row_level_security_expression.head(5)

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 12, Finished, Available, Finished)

  [1/3] Extracting RLS expressions from 'FabricDagsterK1'...
  [2/3] Extracting RLS expressions from 'FabircDagsterK1-Cloud'...
  [3/3] Extracting RLS expressions from 'Dashboard'...
✅ Extracted 0 RLS expression(s)



""


## 8. Data Refresh History Extraction

### Purpose
Extract dataset refresh history for the past 14 days to monitor data freshness and identify failures.

### Output Fields
- **DatasetID**: Semantic model identifier
- **RequestID**: Unique refresh request identifier
- **StartTime/EndTime**: Refresh timing
- **RefreshType**: Full/Incremental/etc.
- **Status**: Completed/Failed/In Progress
- **ExtendedStatus**: Additional status details
- **ServiceExceptionJson**: Error details for failed refreshes
- **ImportTime**: Extraction timestamp

### Business Value
Enables refresh monitoring, SLA tracking, failure alerting, and performance optimization.

In [11]:
data_refresh_request = pd.DataFrame()
total_refreshes = 0

for idx, (_, row) in enumerate(metadata_semantic_model.iterrows(), 1):
    workspace_name = row["WorkspaceName"]
    dataset_name = row["DatasetName"]
    dataset_id = row["DatasetID"]
    
    print(f"  [{idx}/{len(metadata_semantic_model)}] Extracting refresh history for '{dataset_name}'...")
    
    try:
        refreshes = fabric.list_refresh_requests(workspace=workspace_name, dataset=dataset_name)
        
        if len(refreshes) > 0:
            # Filter refreshes within date range
            refreshes = duckdb.query(f'''
                SELECT
                    '{dataset_id}' AS DatasetID,
                    a."Request ID" AS RequestID,
                    a."Start Time" AS StartTime,
                    a."End Time" AS EndTime,
                    a."Refresh Type" AS RefreshType,
                    a."Service Exception Json" AS ServiceExceptionJson,
                    a.Status,
                    CAST(a."Extended Status" AS VARCHAR) AS ExtendedStatus,
                    CAST('{Current_Datetime}' AS TIMESTAMP) AS ImportTime
                FROM refreshes a
                WHERE CAST(a."Start Time" AS DATE) BETWEEN '{Previous_Date}' AND '{Current_Date}' 
            ''').to_df()
            
            data_refresh_request = pd.concat([data_refresh_request, refreshes], ignore_index=True)
            total_refreshes += len(refreshes)
            print(f"    ✓ Found {len(refreshes)} refresh(es)")
            
    except Exception as e:
        print(f"    ⚠️  Failed to retrieve refresh history: {str(e)[:50]}")

print(f"\n✅ Extracted {total_refreshes} refresh record(s) from {Previous_Date} to {Current_Date}\n")
data_refresh_request.head(5)

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 13, Finished, Available, Finished)

  [1/3] Extracting refresh history for 'FabricDagsterK1'...
    ✓ Found 0 refresh(es)
  [2/3] Extracting refresh history for 'FabircDagsterK1-Cloud'...
    ✓ Found 0 refresh(es)
  [3/3] Extracting refresh history for 'Dashboard'...
    ✓ Found 10 refresh(es)

✅ Extracted 10 refresh record(s) from 2025-11-09 to 2025-11-23



,DatasetID,RequestID,StartTime,EndTime,RefreshType,ServiceExceptionJson,Status,ExtendedStatus,ImportTime
0,26ce789d-54a1-4ded-bdc9-de65897d2f8e,c559e7d3-35a9-409a-aedc-d9969f60e3cd,2025-11-22 10:51:28.883000+00:00,2025-11-22 10:51:28.947000+00:00,DirectLakeFraming,None,Completed,None,2025-11-23 21:11:14
1,26ce789d-54a1-4ded-bdc9-de65897d2f8e,bb98f426-bc1e-4997-9aef-e4db731afb88,2025-11-22 10:51:08.883000+00:00,2025-11-22 10:51:10.163000+00:00,DirectLakeFraming,"{""errorCode"":""Premium_ASWL_Error"",""errorDescri...",Failed,None,2025-11-23 21:11:14
2,26ce789d-54a1-4ded-bdc9-de65897d2f8e,c4886c85-3cae-4e0e-8e64-74a78235d159,2025-11-22 10:50:58.880000+00:00,2025-11-22 10:50:59.630000+00:00,DirectLakeFraming,"{""errorCode"":""Premium_ASWL_Error"",""errorDescri...",Failed,None,2025-11-23 21:11:14
3,26ce789d-54a1-4ded-bdc9-de65897d2f8e,702169ae-7665-4d02-a350-6064e7aaec0a,2025-11-22 10:50:48.880000+00:00,2025-11-22 10:50:49.693000+00:00,DirectLakeFraming,"{""errorCode"":""Premium_ASWL_Error"",""errorDescri...",Failed,None,2025-11-23 21:11:14
4,26ce789d-54a1-4ded-bdc9-de65897d2f8e,69fd2d97-cd24-4d3a-ae37-bf13720c6c9b,2025-11-22 10:50:38.880000+00:00,2025-11-22 10:50:40.083000+00:00,DirectLakeFraming,"{""errorCode"":""Premium_ASWL_Error"",""errorDescri...",Failed,None,2025-11-23 21:11:14


## 9. Export Data to Lakehouse

### Purpose
Persist all extracted metadata to Delta Lake tables in Microsoft Fabric Lakehouse for long-term storage, querying, and reporting.

### Features
- **Batch Export**: Writes all 8 metadata DataFrames to separate Delta tables
- **Overwrite Mode**: Refreshes tables completely on each run for idempotency
- **Delta Lake Format**: Leverages ACID transactions, schema evolution, and time travel
- **Optimized Storage**: Automatic compression and partitioning via Spark

### Output Tables

| Table Name | Description | Key Fields |
|------------|-------------|------------|
| `FACT_SemanticModels` | Semantic model inventory | WorkspaceID, DatasetID, DatasetName |
| `FACT_Reports` | Power BI reports catalog | ReportID, DatasetID, WebUrl |
| `FACT_Measures` | DAX measures repository | DatasetID, Measure, Expression |
| `FACT_Relationships` | Table relationship mapping | FromTable, ToTable, Cardinality |
| `FACT_Table_Columns` | Complete schema metadata | TableName, ColumnName, DataType |
| `FACT_RoleMembers` | Security role assignments | RoleKey, RoleName, MemberName |
| `FACT_RLS_Expression` | Row-level security filters | RoleKey, TableName, FilterExpression |
| `FACT_DataRefresh` | 14-day refresh history | DatasetID, Status, StartTime |

### Business Use Cases
1. **Governance Dashboard**: Track all semantic models and their lineage
2. **Security Audit**: Monitor RLS roles and member assignments
3. **Performance Monitoring**: Analyze refresh patterns and failures
4. **Impact Analysis**: Understand report dependencies on datasets
5. **Documentation**: Auto-generate data dictionary from schema metadata

### Technical Notes
- ⚠️ **Prerequisites**: Requires write access to default Lakehouse
- 📊 **Query Access**: Tables can be queried via SQL Endpoint
- 🔄 **Execution**: Run this cell after successful data extraction (cells 1-8)
- 💾 **Persistence**: Data persists across notebook sessions
- 🔍 **Schema**: Column names with spaces will cause errors - clean before export if needed

In [32]:
spark.createDataFrame(metadata_semantic_model).write.mode('overwrite').saveAsTable('FACT_SemanticModels')
spark.createDataFrame(metadata_report).write.mode('overwrite').saveAsTable('FACT_Reports')
spark.createDataFrame(metadata_measures).write.mode('overwrite').saveAsTable('FACT_Measures')
spark.createDataFrame(metadata_relationships).write.mode('overwrite').saveAsTable('FACT_Relationships')
spark.createDataFrame(metadata_table_columns).write.mode('overwrite').saveAsTable('FACT_Table_Columns')
spark.createDataFrame(metadata_role_members).write.mode('overwrite').saveAsTable('FACT_RoleMembers')
spark.createDataFrame(row_level_security_expression).write.mode('overwrite').saveAsTable('FACT_RLS_Expression')
spark.createDataFrame(data_refresh_request).write.mode('overwrite').saveAsTable('FACT_DataRefresh')

StatementMeta(, a88cf86e-dea1-4f0b-8448-815c576e7969, 34, Finished, Available, Finished)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:351: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  [UNSUPPORTED_DATA_TYPE_FOR_ARROW_CONVERSION] uint64 is not supported in conversion to Arrow.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


## Summary & Next Steps

### Data Extraction Complete ✅

The following metadata has been successfully extracted:

1. **Semantic Models**: Dataset inventory and properties
2. **Reports**: Report-to-dataset mappings and URLs
3. **Measures**: DAX calculations and business logic
4. **Tables & Columns**: Complete schema documentation
5. **Relationships**: Data model structure
6. **Roles & Members**: Security role assignments
7. **RLS Expressions**: Row-level security filters
8. **Refresh History**: Data refresh monitoring (14-day window)

### Output DataFrames Available:
- `metadata_semantic_model`
- `metadata_report`
- `metadata_measures`
- `metadata_table_columns`
- `metadata_relationships`
- `metadata_role_members`
- `row_level_security_expression`
- `data_refresh_request`

### Recommended Next Steps:
1. **Export to Data Lake**: Write DataFrames to Delta/Parquet format
2. **Load to Database**: Insert into SQL database for querying
3. **Create Dashboards**: Build monitoring reports in Power BI
4. **Schedule Pipeline**: Automate daily/weekly extraction
5. **Implement Alerts**: Set up notifications for refresh failures

### Sample Export Code:
```python
# Export to Parquet
metadata_semantic_model.to_parquet('semantic_models.parquet')

# Or write to Lakehouse
spark.createDataFrame(metadata_semantic_model).write.mode('overwrite').saveAsTable('metadata.semantic_models')
```
